In [1]:
import pandas as pd

df = pd.read_csv('synthetic_ecommerce_data.csv')
df['No Discount Revenue'] = (df['Revenue'] / (1 - df['Discount_Applied'])).round(2)
df['Lost Revenue'] = (df['No Discount Revenue'] - df['Revenue']).round(2)
df['Per Unit Price'] = (df['No Discount Revenue'] / df['Units_Sold']).round(2)
df['CAC'] = df['Ad_Spend']
df = df.drop(columns=['Ad_Spend'])
df['CAC Percent'] = ((df['CAC'] / df['Revenue']) * 100).round(2)
df['Post CAC Revenue'] = (df['Revenue'] - df['CAC']).round(2)
df['ROAS'] = (df['Revenue'] / df['CAC']).round(2)
prod_revenue = df.groupby('Product_ID')[['Revenue', 'Lost Revenue']].sum().reset_index()
prod_revenue = prod_revenue.rename(columns={'Revenue': 'Total Product Revenue'})
prod_revenue = prod_revenue.rename(columns={'Lost Revenue': 'Total Product Lost Revenue'})
df = df.merge(prod_revenue, on='Product_ID', how='left')
df['Percent of Total Product Revenue'] = ((df['Revenue'] / df['Total Product Revenue']) * 100).round(2)
df['Percent of Total Product Lost Revenue'] = ((df['Lost Revenue'] / df['Total Product Lost Revenue']) * 100).round(2)
LTV = df.groupby('Customer_ID')[['Revenue','Lost Revenue']].sum().reset_index()
LTV = LTV.rename(columns={'Revenue': 'LTV'})
LTV = LTV.rename(columns={'Lost Revenue': 'Lost LTV'})
LTV['Lost LTV Percent'] = ((LTV['Lost LTV'] / LTV['LTV']) * 100).round(2)   
df= df.merge(LTV, on='Customer_ID', how='left')
df['Days Since Transaction'] = df['Transaction_Date'].apply(lambda x: (pd.to_datetime('2024-12-06') - pd.to_datetime(x)).days)
customer_days_since_last_purchase = df.groupby('Customer_ID')[['Days Since Transaction']].min().reset_index()
customer_days_since_last_purchase = customer_days_since_last_purchase.rename(columns={'Days Since Transaction': 'Customer Days Since Last Purchase'})
customer_days_since_first_purchase = df.groupby('Customer_ID')[['Days Since Transaction']].max().reset_index()
customer_days_since_first_purchase = customer_days_since_first_purchase.rename(columns={'Days Since Transaction': 'Customer Days Since First Purchase'})
customer_revenue = df.groupby('Customer_ID')[['Revenue']].sum().reset_index()
customer_revenue = customer_revenue.rename(columns={'Revenue': 'Total Customer Revenue'})
customer_details = df.groupby('Customer_ID')[['Transaction_ID']].count().reset_index()
customer_details = customer_details.rename(columns={'Transaction_ID': 'Total Customer Transactions'})
customer_details = customer_details.merge(customer_days_since_first_purchase, on='Customer_ID', how='left')
customer_details = customer_details.merge(customer_days_since_last_purchase, on='Customer_ID', how='left')
customer_details = customer_details.merge(customer_revenue, on='Customer_ID', how='left')
customer_details['AOV'] = (customer_details['Total Customer Revenue'] / customer_details['Total Customer Transactions']).round(2)
customer_details['Customer Recency'] = customer_details['Customer Days Since Last Purchase'].apply(lambda x: 'Active' if x <= 30 else 'Inactive')
customer_details['Customer Tenure'] = customer_details['Customer Days Since First Purchase'].apply(lambda x: 'New' if x <= 30 else 'Existing')
customer_details['Customer Lifetime'] = customer_details['Customer Days Since First Purchase'] - customer_details['Customer Days Since Last Purchase']
df = df.merge(customer_details, on='Customer_ID', how='left')
df.to_csv('enhanced_synthetic_ecommerce_data.csv', index=False)

In [6]:
df.columns

Index(['Transaction_ID', 'Customer_ID', 'Product_ID', 'Transaction_Date',
       'Units_Sold', 'Discount_Applied', 'Revenue', 'Clicks', 'Impressions',
       'Conversion_Rate', 'Category', 'Region', 'Ad_CTR', 'Ad_CPC',
       'No Discount Revenue', 'Lost Revenue', 'Per Unit Price', 'CAC',
       'CAC Percent', 'Post CAC Revenue', 'ROAS', 'Total Product Revenue',
       'Total Product Lost Revenue', 'Percent of Total Product Revenue',
       'Percent of Total Product Lost Revenue', 'LTV', 'Lost LTV',
       'Lost LTV Percent', 'Days Since Transaction',
       'Total Customer Transactions', 'Customer Days Since First Purchase',
       'Customer Days Since Last Purchase', 'Total Customer Revenue', 'AOV',
       'Customer Recency', 'Customer Tenure', 'Customer Lifetime'],
      dtype='object')